In [2]:

import numpy as np
import csv
import pandas as pd
import miceforest as mf

import joblib
from joblib import load

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import scipy.stats as stats

import importlib
import sys
import feature_sets
importlib.reload(feature_sets)


import model_util
importlib.reload(model_util)

#/usr/bin/python3

<module 'model_util' from '/home/eake191/resmed202100066-Glaucoma_PRS/emma_summer2023/honours/model_util.py'>

# Load data

<br>

### Prevalent:

In [3]:
merged_df = pd.read_pickle('/mnt/shared_folders/eResearch_glaucoma_project/emma_summer2023/honours/data/derived/derived_cols_merged.pkl')


<br>

### Incidence:

In [4]:
# merged_df = pd.read_pickle('/mnt/shared_folders/eResearch_glaucoma_project/emma_summer2023/honours/data/derived/derived_cols_merged.pkl')
odsl_feature_list = feature_sets.ODSL_features['feature'].values
merged_df = pd.read_csv('incidence_merged_df.csv')
merged_df


,f.eid,glaucoma_control,diagnosis_source,age_diagnosed,date_diagnosed,date_attending_assessment_center,time_to_diagnosis,tte_3year,tte_5year,tte_10year,...,Snoring,Daytime sleeping frequency,Exclusion,Glaucoma (prevalent D|TD),IOP subcohort,training_test_split_90_10,Exercise (summed MET minutes per week),training_test_split_80_20,tte_5year_ttsplit,tte_10year_ttsplit
0,1000011,Control,NaN,NaN,NaN,2008-04-22,NaN,Control,Control,Control,...,0.0,0.0,0.0,Control,0,train,1668.0,train,train,train
1,1000026,Control,NaN,NaN,NaN,2009-03-18,NaN,Control,Control,Control,...,1.0,0.0,0.0,Control,0,train,2826.0,train,train,train
2,1000032,Control,NaN,NaN,NaN,2008-04-10,NaN,Control,Control,Control,...,0.0,0.0,0.0,Control,0,train,NaN,test,train,train
3,1000044,Control,NaN,NaN,NaN,2008-09-13,NaN,Control,Control,Control,...,1.0,1.0,0.0,Control,0,train,438.0,train,train,train
4,1000058,Control,NaN,NaN,NaN,2009-02-05,NaN,Control,Control,Control,...,0.0,1.0,0.0,Control,0,train,NaN,train,train,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502414,6024155,Control,NaN,NaN,NaN,2008-04-11,NaN,Control,Control,Control,...,0.0,0.0,0.0,Control,0,test,NaN,test,test,test
502415,6024163,Incident,GP,61.0,2016-04-25,2009-10-03,6.55989,NaN,NaN,Glaucoma,...,0.0,0.0,0.0,Glaucoma,1,train,1935.0,train,NaN,test
502416,6024172,Control,NaN,NaN,NaN,2010-02-10,NaN,Control,Control,Control,...,0.0,0.0,0.0,Control,1,train,NaN,train,train,train
502417,6024181,Control,NaN,NaN,NaN,2008-09-08,NaN,Control,Control,Control,...,1.0,0.0,0.0,Control,0,train,682.0,train,train,train


In [5]:
# Set categorical features

odsl_categorical_features = feature_sets.ODSL_features[feature_sets.ODSL_features['coding_type'].isin(['binary', 'nominal'])]['feature'].values
merged_df[odsl_categorical_features] = merged_df[odsl_categorical_features].astype('category')

In [68]:
IOP_subcohort_df = merged_df[merged_df['IOP subcohort'] == 1] # 112,156 individuals
len(IOP_subcohort_df[IOP_subcohort_df['tte_5year'] == 'Glaucoma'])

474

In [17]:
merged_df[merged_df['tte_10year'] == 'Glaucoma']['Sex'].astype(int).mean()

np.float64(0.47988999656239256)

<br>

# tte: 3 years


In [12]:
# 1. First, separate cases and controls
cases = IOP_subcohort_df[IOP_subcohort_df['tte_3year'] == 'Glaucoma']
controls = IOP_subcohort_df[IOP_subcohort_df['tte_3year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_3year']  # Stratify by glaucoma status (though all are cases)
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_3year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_3year_80_20_split'] = 'train'
test_df['tte_3year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
IOP_subcohort_df = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(IOP_subcohort_df['tte_3year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    IOP_subcohort_df['tte_3year_80_20_split'],
    IOP_subcohort_df['tte_3year'],
    normalize='index'
))

Final Split Proportions:
tte_3year_80_20_split
train    0.799985
test     0.200015
Name: proportion, dtype: float64

Case/Control Distribution in Each Split:
tte_3year               Control  Glaucoma
tte_3year_80_20_split                    
test                   0.997792  0.002208
train                  0.997837  0.002163


In [16]:
IOP_subcohort_df.value_counts

<bound method DataFrame.value_counts of           f.eid glaucoma_control  diagnosis_source  age_diagnosed  \
422596  5225971         Incident         Inpatient           55.0   
258478  3584796         Incident    GP & Inpatient           65.0   
26399   1264007         Incident                GP           68.0   
310582  4105835         Incident  Self-report & GP           68.0   
184753  2847542         Incident    GP & Inpatient           58.0   
...         ...              ...               ...            ...   
118016  2180173          Control               NaN            NaN   
91122   1911234          Control               NaN            NaN   
111416  2114177          Control               NaN            NaN   
142769  2427706          Control               NaN            NaN   
274130  3741318          Control               NaN            NaN   

       date_diagnosed date_attending_assessment_center  time_to_diagnosis  \
422596     2011-09-21                       2010-02-05

## Missing feature stats

In [16]:
IOP_subcohort_df['tte_3year']

5         Control
7         Control
9         Control
20        Control
22        Control
           ...   
502406    Control
502409    Control
502410    Control
502415        NaN
502416    Control
Name: tte_3year, Length: 112156, dtype: object

In [4]:
glaucoma_df_3year_tte = IOP_subcohort_df[IOP_subcohort_df['tte_3year'] == 'Glaucoma']
control_df_3year_tte = IOP_subcohort_df[IOP_subcohort_df['tte_3year'] == 'Control']

total_n = len(IOP_subcohort_df)
glaucoma_n = len(glaucoma_df_3year_tte)
control_n = len(control_df_3year_tte)

missing_feature_df_3year_tte = pd.DataFrame(columns=[
    'Feature',
    'N missing', # used for sorting
    'N missing (%)',
    'N missing, glaucoma (%)',
    'N missing, control (%)',
    'p',
])
missing_feature_df_3year_tte = missing_feature_df_3year_tte.set_index('Feature', drop=True)

for feature in odsl_feature_list:
    n_missing = IOP_subcohort_df[feature].isna().sum()
    n_missing_percent = (n_missing / total_n) * 100
    missing_feature_df_3year_tte.loc[feature, 'N missing'] = n_missing
    if n_missing >= 10000:
        missing_feature_df_3year_tte.loc[feature, 'N missing (%)'] = f'{n_missing:,} ({n_missing_percent:0.2f}%)'
    else:
        missing_feature_df_3year_tte.loc[feature, 'N missing (%)'] = f'{n_missing} ({n_missing_percent:0.2f}%)'

    # Glaucoma
    n_missing_glaucoma = glaucoma_df_3year_tte[feature].isna().sum()
    n_missing_percent = (n_missing_glaucoma / glaucoma_n) * 100
    if n_missing >= 10000:
        missing_feature_df_3year_tte.loc[feature, 'N missing, glaucoma (%)'] = f'{n_missing_glaucoma:,} ({n_missing_percent:0.2f}%)'
    else:
        missing_feature_df_3year_tte.loc[feature, 'N missing, glaucoma (%)'] = f'{n_missing_glaucoma} ({n_missing_percent:0.2f}%)'

    # Control
    n_missing_control = control_df_3year_tte[feature].isna().sum()
    n_missing_percent = (n_missing_control / control_n) * 100
    if n_missing >= 10000:
        missing_feature_df_3year_tte.loc[feature, 'N missing, control (%)'] = f'{n_missing_control:,} ({n_missing_percent:0.2f}%)'
    else:
        missing_feature_df_3year_tte.loc[feature, 'N missing, control (%)'] = f'{n_missing_control} ({n_missing_percent:0.2f}%)'

    # Chi-square

    if n_missing_glaucoma == 0 and n_missing_control == 0:
        continue

    # contignency tab
    ctb = pd.crosstab(index=IOP_subcohort_df[feature].isna() == True, columns=IOP_subcohort_df['Glaucoma (prevalent D|TD)'])
    
    p_val = stats.chi2_contingency(ctb).pvalue
    if p_val < 0.001:
        p_val = '<0.001'
    else:
        p_val = f'{p_val:0.3f}'
    missing_feature_df_3year_tte.loc[feature, 'p'] = p_val

missing_feature_df_3year_tte = missing_feature_df_3year_tte.sort_values(by='N missing', axis=0, ascending=False)
missing_feature_df_3year_tte = missing_feature_df_3year_tte.drop(columns=['N missing'])

NameError: name 'IOP_subcohort_df' is not defined

In [20]:
missing_feature_df_3year_tte.to_csv('./data/imputed/missing_feature_count_3year_tte.tsv', sep='\t')

In [21]:
missing_feature_df_3year_tte.to_html('./data/imputed/missing_feature_count_3year_tte.html')

In [22]:
missing_feature_df_3year_tte.head(25)

,N missing (%),"N missing, glaucoma (%)","N missing, control (%)",p
Feature,,,,
Exercise (summed MET minutes per week),"24,704 (22.03%)",55 (23.31%),"23,773 (21.92%)",<0.001
Total household income,"16,339 (14.57%)",37 (15.68%),"15,681 (14.46%)",<0.001
Plasma oestradiol,"15,437 (13.76%)",36 (15.25%),"14,909 (13.75%)",0.487
Plasma glucose,"15,351 (13.69%)",33 (13.98%),"14,814 (13.66%)",0.203
HDL,"15,283 (13.63%)",33 (13.98%),"14,747 (13.60%)",0.182
Plasma albumin,"15,205 (13.56%)",34 (14.41%),"14,672 (13.53%)",0.187
Plasma Vitamin D,"12,980 (11.57%)",29 (12.29%),"12,534 (11.56%)",0.464
HbA1c,"10,086 (8.99%)",30 (12.71%),"9,721 (8.97%)",0.078
Plasma testosterone,9252 (8.25%),22 (9.32%),8919 (8.23%),0.131


## Run imputation

In [18]:
X_train_3year_tte, y_train_3year_tte, X_test_3year_tte, y_test_3year_tte = model_util.get_train_test_datasets(
    IOP_subcohort_df, 
    'tte_3year_80_20_split', 
    'tte_3year', 
    odsl_feature_list
)

In [38]:
imputation_kernel_3year_tte = mf.ImputationKernel(
    X_train_3year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending',
)

<br>

Below cell changed iterations from 20 to 13 because kernel started to die each time running

In [39]:
imputation_kernel_3year_tte.mice(
    verbose=True,
    iterations=13, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

Initialized logger with name MICE Iterations 1 - 13 and 4 levels
1 Dataset 0
 | Exercise (summed MET minutes per week) | Total household income | Plasma oestradiol | Plasma glucose | HDL | Plasma albumin | Plasma Vitamin D | HbA1c | Plasma testosterone | Snoring | Past smoking frequency | Plasma total bilirubin | LDL | C-reactive protein | Triglycerides | Plasma urate | eGFR serum creatinine | Total cholesterol | Hearing difficulty (self-reported) | Diet score | Systemic immune inflammation index | Urinary sodium-creatinine ratio | Albumin-creatinine ratio | Polygenic risk score | Speech reception threshold | Tinnitus frequency (self-reported) | Corneal hysteresis inter-eye difference | IOPg pre-treatment inter-eye difference | Education | PM2.5 exposure | Private healthcare utilisation | Arterial stiffness index | Spherical equivalent | Urban residence | Daytime sleeping frequency | Normal sleep duration | Vitamin C supplementation | Multivitamin supplementation | Ethnicity | Poor ora

In [40]:
joblib.dump(imputation_kernel_3year_tte, './data/imputed/imputation_kernel_13iter_3year_tte.pkl')

['./data/imputed/imputation_kernel_13iter_3year_tte.pkl']

## Save imputed data & train/test

In [41]:
imputation_kernel_3year_tte = load('./data/imputed/imputation_kernel_13iter_3year_tte.pkl')

In [94]:
X_train_imputed_3year_tte = imputation_kernel_3year_tte.complete_data().reset_index(drop=True)
X_test_imputed_3year_tte = imputation_kernel_3year_tte.impute_new_data(X_test_3year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_3year_tte, './data/imputed/IOPsubcohort_X_train_imputed_3year_tte.pkl')
joblib.dump(X_test_imputed_3year_tte, './data/imputed/IOPsubcohort_X_test_imputed_3year_tte.pkl')

joblib.dump(y_train_3year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_3year_tte.pkl')
joblib.dump(y_test_3year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_3year_tte.pkl')

KeyboardInterrupt: 

In [ ]:
X_merged_imputed_3year_tte = pd.concat((X_train_imputed_3year_tte, X_test_imputed_3year_tte), ignore_index=True)
y_merged_3year_tte = pd.concat((y_train_3year_tte, y_test_3year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_3year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_3year_tte.pkl')
joblib.dump(y_merged_3year_tte, './data/imputed/IOPsubcohort_y_merged_3year_tte.pkl')

In [44]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_3year_tte)

X_train_scaled_3year_tte = pd.DataFrame(scaler.transform(X_train_imputed_3year_tte), columns=X_train_3year_tte.columns)
X_test_scaled_3year_tte = pd.DataFrame(scaler.transform(X_test_imputed_3year_tte), columns=X_test_3year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_3year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_3year_tte.pkl')
joblib.dump(X_test_scaled_3year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_3year_tte.pkl')

['./data/imputed/IOPsubcohort_X_test_imputed_scaled_3year_tte.pkl']

<br>

<br>

# tte: 5 years


In [69]:
# 1. First, separate cases and controls
cases = IOP_subcohort_df[IOP_subcohort_df['tte_5year'] == 'Glaucoma']
controls = IOP_subcohort_df[IOP_subcohort_df['tte_5year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_5year']  # Stratify by glaucoma status 
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_5year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_5year_80_20_split'] = 'train'
test_df['tte_5year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
IOP_subcohort_df = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(IOP_subcohort_df['tte_5year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    IOP_subcohort_df['tte_5year_80_20_split'],
    IOP_subcohort_df['tte_5year'],
    normalize='index'
))

Final Split Proportions:
tte_5year_80_20_split
train    0.799991
test     0.200009
Name: proportion, dtype: float64

Case/Control Distribution in Each Split:
tte_5year               Control  Glaucoma
tte_5year_80_20_split                    
test                   0.995639  0.004361
train                  0.995650  0.004350


In [73]:
IOP_subcohort_df['tte_5year'].value_counts()

tte_5year
Control     108431
Glaucoma       474
Name: count, dtype: int64

In [75]:
X_train_5year_tte, y_train_5year_tte, X_test_5year_tte, y_test_5year_tte = model_util.get_train_test_datasets(
    IOP_subcohort_df, 
    'tte_5year_80_20_split', 
    'tte_5year', 
    odsl_feature_list )

In [104]:
print(len(X_train_5year_tte))
print(len(y_train_5year_tte))
print(len(X_test_5year_tte))
print(len(y_test_5year_tte))

87123
87123
21782
21782


In [105]:
print("=== TRAINING DATA ===")
print(f"Total training samples: {len(y_train_5year_tte)}")
print(f"Training controls (y=0): {sum(y_train_5year_tte == 0)}")
print(f"Training cases (y=1): {sum(y_train_5year_tte == 1)}")
print(f"Training case prevalence: {sum(y_train_5year_tte == 1)/len(y_train_5year_tte):.3%}")

# Check the test data
print("\n=== TEST DATA ===")
print(f"Total test samples: {len(y_test_5year_tte)}")
print(f"Test controls (y=0): {sum(y_test_5year_tte == 0)}")
print(f"Test cases (y=1): {sum(y_test_5year_tte == 1)}")
print(f"Test case prevalence: {sum(y_test_5year_tte == 1)/len(y_test_5year_tte):.3%}")

=== TRAINING DATA ===
Total training samples: 87123
Training controls (y=0): 86744
Training cases (y=1): 379
Training case prevalence: 0.435%

=== TEST DATA ===
Total test samples: 21782
Test controls (y=0): 21687
Test cases (y=1): 95
Test case prevalence: 0.436%


## Run imputation

In [106]:
imputation_kernel_5year_tte = mf.ImputationKernel(
    X_train_5year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending', 
)

In [107]:
imputation_kernel_5year_tte.mice(
    verbose=True,
    iterations=13, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

joblib.dump(imputation_kernel_5year_tte, './data/imputed/imputation_kernel_13iter_5year_tte.pkl')

Initialized logger with name MICE Iterations 1 - 13 and 4 levels
1 Dataset 0
 | Exercise (summed MET minutes per week) | Total household income | Plasma oestradiol | Plasma glucose | HDL | Plasma albumin | Plasma Vitamin D | HbA1c | Plasma testosterone | Snoring | Past smoking frequency | Plasma total bilirubin | LDL | C-reactive protein | Triglycerides | Plasma urate | eGFR serum creatinine | Total cholesterol | Hearing difficulty (self-reported) | Diet score | Systemic immune inflammation index | Urinary sodium-creatinine ratio | Albumin-creatinine ratio | Polygenic risk score | Speech reception threshold | Tinnitus frequency (self-reported) | Corneal hysteresis inter-eye difference | IOPg pre-treatment inter-eye difference | Education | PM2.5 exposure | Private healthcare utilisation | Arterial stiffness index | Spherical equivalent | Urban residence | Daytime sleeping frequency | Normal sleep duration | Vitamin C supplementation | Multivitamin supplementation | Ethnicity | Poor ora

['./data/imputed/imputation_kernel_13iter_5year_tte.pkl']

<br>

## Save imputed data & train/test

In [98]:
imputation_kernel_5year_tte = load('./data/imputed/imputation_kernel_13iter_5year_tte.pkl')

In [108]:
X_train_imputed_5year_tte = imputation_kernel_5year_tte.complete_data().reset_index(drop=True)
X_test_imputed_5year_tte = imputation_kernel_5year_tte.impute_new_data(X_test_5year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_5year_tte, './data/imputed/IOPsubcohort_X_train_imputed_5year_tte.pkl')
joblib.dump(X_test_imputed_5year_tte, './data/imputed/IOPsubcohort_X_test_imputed_5year_tte.pkl')

joblib.dump(y_train_5year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_5year_tte.pkl')
joblib.dump(y_test_5year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_5year_tte.pkl')

['./data/imputed/IOPsubcohort_y_test_5year_tte.pkl']

In [109]:
X_merged_imputed_5year_tte = pd.concat((X_train_imputed_5year_tte, X_test_imputed_5year_tte), ignore_index=True)
y_merged_5year_tte = pd.concat((y_train_5year_tte, y_test_5year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_5year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_5year_tte.pkl')
joblib.dump(y_merged_5year_tte, './data/imputed/IOPsubcohort_y_merged_5year_tte.pkl')

['./data/imputed/IOPsubcohort_y_merged_5year_tte.pkl']

In [110]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_5year_tte)

X_train_scaled_5year_tte = pd.DataFrame(scaler.transform(X_train_imputed_5year_tte), columns=X_train_5year_tte.columns)
X_test_scaled_5year_tte = pd.DataFrame(scaler.transform(X_test_imputed_5year_tte), columns=X_test_5year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_5year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_5year_tte.pkl')
joblib.dump(X_test_scaled_5year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_5year_tte.pkl')

['./data/imputed/IOPsubcohort_X_test_imputed_scaled_5year_tte.pkl']

In [111]:
print("=== AFTER IMPUTATION & SCALING ===")
print(f"Total training samples: {len(y_train_5year_tte)}")
print(f"Training controls (y=0): {sum(y_train_5year_tte == 0)}")
print(f"Training cases (y=1): {sum(y_train_5year_tte == 1)}")
len(X_train_scaled_5year_tte) # 86932, should be 87123

=== AFTER IMPUTATION & SCALING ===
Total training samples: 87123
Training controls (y=0): 86744
Training cases (y=1): 379


87123

<br>

<br>

# tte: 10 years

In [55]:
# 1. First, separate cases and controls
cases = IOP_subcohort_df[IOP_subcohort_df['tte_10year'] == 'Glaucoma']
controls = IOP_subcohort_df[IOP_subcohort_df['tte_10year'] == 'Control']

# 2. Split cases into train/test (80/20)
case_train, case_test = train_test_split(
    cases,
    test_size=0.2,
    random_state=42,
    stratify=cases['tte_10year']  # Stratify by glaucoma status (though all are cases)
)

# 3. For controls, randomly sample to match the case split ratio
# (Ensure controls are independent of cases since no matched IDs exist)
control_train, control_test = train_test_split(
    controls,
    test_size=0.2,
    random_state=42,
    stratify=controls['tte_10year']  # Stratify by control status
)

# 4. Combine and label splits
train_df = pd.concat([case_train, control_train])
test_df = pd.concat([case_test, control_test])

train_df['tte_10year_80_20_split'] = 'train'
test_df['tte_10year_80_20_split'] = 'test'

# 5. Merge back into original DataFrame
IOP_subcohort_df = pd.concat([train_df, test_df])

# 6. QC Check
print("Final Split Proportions:")
print(IOP_subcohort_df['tte_10year_80_20_split'].value_counts(normalize=True))

print("\nCase/Control Distribution in Each Split:")
print(pd.crosstab(
    IOP_subcohort_df['tte_10year_80_20_split'],
    IOP_subcohort_df['tte_10year'],
    normalize='index'
))

Final Split Proportions:
tte_10year_80_20_split
train    0.799985
test     0.200015
Name: proportion, dtype: float64

Case/Control Distribution in Each Split:
tte_10year               Control  Glaucoma
tte_10year_80_20_split                    
test                    0.997792  0.002208
train                   0.997837  0.002163


In [56]:
X_train_10year_tte, y_train_10year_tte, X_test_10year_tte, y_test_10year_tte = model_util.get_train_test_datasets(
    IOP_subcohort_df, 
    'tte_10year_80_20_split', 
    'tte_10year', 
    odsl_feature_list
)

## Run imputation

In [57]:
imputation_kernel_10year_tte = mf.ImputationKernel(
    X_train_10year_tte,
    num_datasets=1,
    random_state=2024,
    mean_match_strategy='normal',
    mean_match_candidates=10,
    save_all_iterations_data=True,
    imputation_order='descending',
)

In [58]:
imputation_kernel_10year_tte.mice(
    verbose=True,
    iterations=13, 

    # LGBM parameters 
    n_estimators=200,
    max_bin=512,
    # max_depth=10,
    # num_leaves=1023,
    # learning_rate=0.1,
)

Initialized logger with name MICE Iterations 1 - 13 and 4 levels
1 Dataset 0
 | Exercise (summed MET minutes per week) | Total household income | Plasma oestradiol | Plasma glucose | HDL | Plasma albumin | Plasma Vitamin D | HbA1c | Plasma testosterone | Snoring | Past smoking frequency | Plasma total bilirubin | LDL | C-reactive protein | Triglycerides | eGFR serum creatinine | Plasma urate | Total cholesterol | Hearing difficulty (self-reported) | Diet score | Systemic immune inflammation index | Urinary sodium-creatinine ratio | Albumin-creatinine ratio | Polygenic risk score | Speech reception threshold | Tinnitus frequency (self-reported) | Corneal hysteresis inter-eye difference | IOPg pre-treatment inter-eye difference | Education | PM2.5 exposure | Private healthcare utilisation | Arterial stiffness index | Urban residence | Spherical equivalent | Daytime sleeping frequency | Normal sleep duration | Vitamin C supplementation | Multivitamin supplementation | Ethnicity | Poor ora

In [59]:
joblib.dump(imputation_kernel_10year_tte, './data/imputed/imputation_kernel_13iter_10year_tte.pkl')

['./data/imputed/imputation_kernel_13iter_10year_tte.pkl']

## Save imputed data & train/test

In [60]:
imputation_kernel_10year_tte = load('./data/imputed/imputation_kernel_13iter_10year_tte.pkl')

In [61]:
X_train_imputed_10year_tte = imputation_kernel_10year_tte.complete_data().reset_index(drop=True)
X_test_imputed_10year_tte = imputation_kernel_10year_tte.impute_new_data(X_test_10year_tte).complete_data().reset_index(drop=True)

joblib.dump(X_train_imputed_10year_tte, './data/imputed/IOPsubcohort_X_train_imputed_10year_tte.pkl')
joblib.dump(X_test_imputed_10year_tte, './data/imputed/IOPsubcohort_X_test_imputed_10year_tte.pkl')

joblib.dump(y_train_10year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_train_10year_tte.pkl')
joblib.dump(y_test_10year_tte.reset_index(drop=True), './data/imputed/IOPsubcohort_y_test_10year_tte.pkl')

['./data/imputed/IOPsubcohort_y_test_10year_tte.pkl']

In [62]:
X_merged_imputed_10year_tte = pd.concat((X_train_imputed_10year_tte, X_test_imputed_10year_tte), ignore_index=True)
y_merged_10year_tte = pd.concat((y_train_10year_tte, y_test_10year_tte), ignore_index=True)

joblib.dump(X_merged_imputed_10year_tte, './data/imputed/IOPsubcohort_X_merged_imputed_10year_tte.pkl')
joblib.dump(y_merged_10year_tte, './data/imputed/IOPsubcohort_y_merged_10year_tte.pkl')

['./data/imputed/IOPsubcohort_y_merged_10year_tte.pkl']

In [63]:
# Apply scaling

scaler = MinMaxScaler()
scaler.fit(X_train_imputed_10year_tte)

X_train_scaled_10year_tte = pd.DataFrame(scaler.transform(X_train_imputed_10year_tte), columns=X_train_10year_tte.columns)
X_test_scaled_10year_tte = pd.DataFrame(scaler.transform(X_test_imputed_10year_tte), columns=X_test_10year_tte.columns)

joblib.dump(scaler, './data/imputed/min_max_scaler.pkl')
joblib.dump(X_train_scaled_10year_tte, './data/imputed/IOPsubcohort_X_train_imputed_scaled_10year_tte.pkl')
joblib.dump(X_test_scaled_10year_tte, './data/imputed/IOPsubcohort_X_test_imputed_scaled_10year_tte.pkl')

['./data/imputed/IOPsubcohort_X_test_imputed_scaled_10year_tte.pkl']

<br>

<br>

# RNFL models:

In [ ]:
import feature_sets_rnfl
importlib.reload(feature_sets_rnfl)